In [ ]:
import pandas as pd
import numpy as np
import joblib

In [ ]:
def copyWeatherDataToHistory(count):
    weatherForcastingDataFromInternet = pd.read_csv('weather_data.csv')
    ## The weather forcasting data need to be obtained from internet.
    ## Here is just an example that we use the demo weather data from a local city instead.  :)
    
    predictWeather = pd.read_csv('weather_predict.csv')
    historyWeather = pd.read_csv('weather_history.csv')

    historyWeather = historyWeather.append(predictWeather[0:1], ignore_index = True)
    predictWeather = weatherForcastingDataFromInternet[count:count+336]
    
    historyWeather.to_csv('weather_history.csv', index = False)
    predictWeather.to_csv('weather_predict.csv', index = False)

In [ ]:
def copyScheduleDataToHistory():
    schedule = pd.read_csv('schedule.csv')
    historySchedule = pd.read_csv('schedule_history.csv')

    historySchedule = historySchedule.append(schedule[0:6], ignore_index = True)
    
    historySchedule.to_csv('schedule_history.csv', index = False)

In [ ]:
def predictWeatherBasedData():
    weather_data = pd.read_csv("weather_predict.csv")
    ## import the trained machine learning model, that will be used to predict energy_power based on weather data
    pv_model = joblib.load('solar_power_model.sav')
    wd_model = joblib.load('wind_power_model.sav')
    cp_model = joblib.load('consumption_power_model.sav')
    index = []
    names = []
    types = []
    dTime = []
    energy_power = []
    for hour in range(0, 14 * 24):
        for i in range(1, 4):
            index.append(1000 + i)
            dTime.append(weather_data['dTime'][hour])
            if i == 1 :
                names.append("Photovoltaics Tech")
                types.append("photovoltaics")
                energy_power.append(pv_model.predict([[weather_data["temperature"][hour],
                                                       weather_data["light_intensity"][hour],
                                                       weather_data["humidity"][hour],
                                                       weather_data["wind_speed"][hour]]])[0][0])
            if i == 2 :
                names.append("Giant Wind Power")
                types.append("wind_power")
                energy_power.append(wd_model.predict([[weather_data["temperature"][hour],
                                                       weather_data["light_intensity"][hour],
                                                       weather_data["humidity"][hour],
                                                       weather_data["wind_speed"][hour]]])[0][0])
            if i == 3 :
                names.append("City consumption")
                types.append("consumption")
                energy_power.append(cp_model.predict([[weather_data["temperature"][hour],
                                                       weather_data["light_intensity"][hour],
                                                       weather_data["humidity"][hour],
                                                       weather_data["wind_speed"][hour]]])[0][0])
    weather_based_nodes = pd.DataFrame(columns = ['id', 'name', 'type', 'dTime', 'energy_power'])
    weather_based_nodes['id'] = index
    weather_based_nodes['name'] = names
    weather_based_nodes['type'] = types
    weather_based_nodes['dTime'] = dTime
    weather_based_nodes['energy_power'] = energy_power
    weather_based_nodes.to_csv("weather_based_nodes.csv", index = False)

In [ ]:
def Weibull_reliability_function(x) :
        return pow(2.718281828459,-pow((x - 9.6)/213,4.2))
    
def updateControlledNodes():
    weatherNodes = pd.read_csv("weather_based_nodes.csv")
    totalEnergyPower = [0] * (14 * 24)
    length = weatherNodes.shape[0]
    zeroRank = int(weatherNodes['dTime'][0][6:8]) * 24 + int(weatherNodes['dTime'][0][9:-3])
    for row in range(0, length) :
        rank = int(weatherNodes['dTime'][row][6:8]) * 24 + int(weatherNodes['dTime'][row][9:-3]) - zeroRank
        totalEnergyPower[rank] = totalEnergyPower[rank] + weatherNodes['energy_power'][row]
    
    updTotalEnergyPower = []
    for hour in range(0, 14 * 24) :
        updTotalEnergyPower.append(totalEnergyPower[hour] * Weibull_reliability_function(hour))

    sumEnergyPower = []
    sumEnergyPower.append(updTotalEnergyPower[0])
    for hour in range(1, 14 * 24) :
        sumEnergyPower.append(sumEnergyPower[hour - 1] + updTotalEnergyPower[hour])
        
    ## parameters of Storage system (from files uploaded by clients)
    currentStorage = 150
    highLimitStorage = 200
    lowLimitStorage = 100
    
    ## energyChange of Storage system
    storageEnergyChange = []
    tempStorage = currentStorage
    for hour in range(0, 14 * 24) :
        energyChange = updTotalEnergyPower[hour]
        if energyChange > highLimitStorage - tempStorage :
            energyChange = highLimitStorage - tempStorage
        storageEnergyChange.append(float(energyChange))
        tempStorage = tempStorage + energyChange
    
    ## update fossil fuel power needed
    tempStorage = currentStorage
    maxNeedFossilFuelPower = 0
    for hour in range(0, 14 * 24) :
        tempStorage = tempStorage + updTotalEnergyPower[hour]
        if tempStorage > highLimitStorage :
            tempStorage = highLimitStorage
        if tempStorage < lowLimitStorage :
            fossilFuelPower = (lowLimitStorage - tempStorage) / hour
            if fossilFuelPower >= maxNeedFossilFuelPower :
                maxNeedFossilFuelPower = fossilFuelPower
    fossilFuelPowerNeed = [float(maxNeedFossilFuelPower)] * (14 * 24)
    
    ## export to Schedule.csv
    dTime = []
    index = []
    energy_power = []
    firstDay = int(weatherNodes['dTime'][0][0:8])
    firstHour = int(weatherNodes['dTime'][0][9:-3])
    for day in range(0, 14) : 
        for hour in range(0, 24) :
            nowDay = firstDay + day
            nowHour = firstHour + hour
            for i in range(1, 6) :
                if nowHour >= 10 : 
                    dTime.append("{}-{}:00".format(nowDay, nowHour))
                else :
                    dTime.append("{}-0{}:00".format(nowDay, nowHour))
                if i <= 3 :
                    index.append(1000 + i)
                    energy_power.append(weatherNodes['energy_power'][3 * (day * 24 + hour) + i - 1])
                elif i == 4 :
                    index.append(2000 + i - 4)
                    energy_power.append(storageEnergyChange[day * 24 + hour])
                else :
                    index.append(2000 + i - 4)
                    energy_power.append(fossilFuelPowerNeed[day * 24 + hour])
    Schedule = pd.DataFrame(columns = ['dTime', 'id', 'energy_power'])
    Schedule['dTime'] = dTime
    Schedule['id'] = index
    Schedule['energy_power'] = energy_power
    Schedule.to_csv("schedule.csv", index = False)
    
    ## export to Controlled_Nodes.csv
    dTime = []
    index = []
    names = []
    types = []
    energy_power = []
    for day in range(0, 14) :
        for hour in range(0, 24) :
            nowDay = firstDay + day
            nowHour = firstHour + hour
            for i in range(1, 3) :
                if nowHour >= 10 : 
                    dTime.append("{}-{}:00".format(nowDay, nowHour))
                else :
                    dTime.append("{}-0{}:00".format(nowDay, nowHour))
                index.append(2000 + i)
                if i == 1 :
                    types.append("storage")
                    names.append("Intelligent Storage System")
                    energy_power.append(storageEnergyChange[day * 24 + hour])
                else :
                    types.append("fossil_fuel")
                    names.append("Clean Fuel Home")
                    energy_power.append(fossilFuelPowerNeed[day * 24 + hour])
    Controlled_Nodes = pd.DataFrame(columns = ['id', 'name', 'type', 'dTime', 'energy_power'])
    Controlled_Nodes['id'] = index
    Controlled_Nodes['name'] = names
    Controlled_Nodes['type'] = types
    Controlled_Nodes['dTime'] = dTime
    Controlled_Nodes['energy_power'] = energy_power
    Controlled_Nodes.to_csv("controlled_nodes.csv", index = False)

In [ ]:
copyWeatherDataToHistory(1)
copyScheduleDataToHistory()
predictWeatherBasedData()
updateControlledNodes()